In [1]:
# Initial imports.
import sqlalchemy
import numpy as np
from pathlib import Path
from collections import Counter
import pandas as pd
from path import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from collections import Counter

In [2]:
# Loading data
file_path = Path("../Data/final_table_ml_stage2.csv")
risk_df = pd.read_csv(file_path)
risk_df.head()

,business_postal_code_1,inspection_id,inspection_date,inspection_score,violation_description,risk_category,neighborhoods,Current Police Districts,population,avg_income
0,94133,835_20180917,September,88,Improper food storage,Low Risk,107,6,26827.0,40990.0
1,94108,905_20190415,April,87,High risk vermin infestation,High Risk,19,6,13716.0,31542.0
2,94118,1203_20170803,August,77,Moderate risk food holding temperature,Moderate Risk,5,8,38939.0,61609.0
3,94109,1345_20170928,September,81,Improper cooling methods,High Risk,105,4,56322.0,43444.0
4,94114,1352_20180620,June,74,Non service animal,Low Risk,38,3,30574.0,75727.0


In [3]:
# Drop inspection_id
risk_df1 = risk_df.drop(columns='inspection_id')

In [4]:
#drop inspection_score
risk_df2 = risk_df1.drop(columns='inspection_score')
risk_df2

,business_postal_code_1,inspection_date,violation_description,risk_category,neighborhoods,Current Police Districts,population,avg_income
0,94133,September,Improper food storage,Low Risk,107,6,26827.0,40990.0
1,94108,April,High risk vermin infestation,High Risk,19,6,13716.0,31542.0
2,94118,August,Moderate risk food holding temperature,Moderate Risk,5,8,38939.0,61609.0
3,94109,September,Improper cooling methods,High Risk,105,4,56322.0,43444.0
4,94114,June,Non service animal,Low Risk,38,3,30574.0,75727.0
...,...,...,...,...,...,...,...,...
20239,94118,October,Improper storage of equipment utensils or linens,Low Risk,5,8,38939.0,61609.0
20240,94112,April,Moderate risk food holding temperature,Moderate Risk,90,9,73104.0,57629.0
20241,94103,July,Inadequate warewashing facilities or equipment,Low Risk,53,3,23016.0,31131.0
20242,94103,January,Improper cooling methods,High Risk,32,1,23016.0,31131.0


In [5]:
# Create our features
X = risk_df2.drop(columns='risk_category')

# Create our target
y = risk_df2['risk_category']
y.value_counts()

Low Risk         9820
Moderate Risk    7599
High Risk        2825
Name: risk_category, dtype: int64

In [6]:
X.describe()

,business_postal_code_1,neighborhoods,Current Police Districts,population,avg_income
count,20244.000000,20244.000000,20244.000000,20244.000000,20244.000000
mean,94114.061203,57.263485,5.399081,37738.252420,51372.542482
std,9.350442,35.014539,2.681820,21784.166592,17502.372234
min,94102.000000,1.000000,1.000000,374.000000,14609.000000
25%,94108.000000,28.000000,3.000000,23016.000000,40990.000000
50%,94111.000000,53.000000,6.000000,33115.000000,53795.000000
75%,94121.000000,96.000000,8.000000,55492.000000,61362.000000
max,94134.000000,117.000000,10.000000,74633.000000,95313.000000


In [7]:
# encode Month as a number
months_num = {
   "January": 1,
   "February": 2,
   "March": 3,
   "April": 4,
   "May": 5,
   "June": 6,
   "July": 7,
   "August": 8,
   "September": 9,
   "October": 10,
   "November": 11,
   "December": 12,
}

In [8]:
X["months_num"] = X["inspection_date"].apply(lambda x: months_num[x])

In [9]:
months_num["June"]

6

In [10]:
X

,business_postal_code_1,inspection_date,violation_description,neighborhoods,Current Police Districts,population,avg_income,months_num
0,94133,September,Improper food storage,107,6,26827.0,40990.0,9
1,94108,April,High risk vermin infestation,19,6,13716.0,31542.0,4
2,94118,August,Moderate risk food holding temperature,5,8,38939.0,61609.0,8
3,94109,September,Improper cooling methods,105,4,56322.0,43444.0,9
4,94114,June,Non service animal,38,3,30574.0,75727.0,6
...,...,...,...,...,...,...,...,...
20239,94118,October,Improper storage of equipment utensils or linens,5,8,38939.0,61609.0,10
20240,94112,April,Moderate risk food holding temperature,90,9,73104.0,57629.0,4
20241,94103,July,Inadequate warewashing facilities or equipment,53,3,23016.0,31131.0,7
20242,94103,January,Improper cooling methods,32,1,23016.0,31131.0,1


In [11]:
# drop inspection_date
X = X.drop(columns='inspection_date')
X.head()

,business_postal_code_1,violation_description,neighborhoods,Current Police Districts,population,avg_income,months_num
0,94133,Improper food storage,107,6,26827.0,40990.0,9
1,94108,High risk vermin infestation,19,6,13716.0,31542.0,4
2,94118,Moderate risk food holding temperature,5,8,38939.0,61609.0,8
3,94109,Improper cooling methods,105,4,56322.0,43444.0,9
4,94114,Non service animal,38,3,30574.0,75727.0,6


In [12]:
# encode features
X_encoded = pd.get_dummies(X)
X_encoded

,business_postal_code_1,neighborhoods,Current Police Districts,population,avg_income,months_num,violation_description_Consumer advisory not provided for raw or undercooked foods,violation_description_Contaminated or adulterated food,violation_description_Discharge from employee nose mouth or eye,violation_description_Employee eating or smoking,...,violation_description_Unauthorized or unsafe use of time as a public health control measure,violation_description_Unclean hands or improper use of gloves,violation_description_Unclean nonfood contact surfaces,violation_description_Unclean or degraded floors walls or ceilings,violation_description_Unclean or unsanitary food contact surfaces,violation_description_Unclean unmaintained or improperly constructed toilet facilities,violation_description_Unpermitted food facility,violation_description_Unsanitary employee garments hair or nails,violation_description_Wiping cloths not clean or properly stored or inadequate sanitizer,violation_description_Worker safety hazards
0,94133,107,6,26827.0,40990.0,9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,94108,19,6,13716.0,31542.0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,94118,5,8,38939.0,61609.0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,94109,105,4,56322.0,43444.0,9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,94114,38,3,30574.0,75727.0,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20239,94118,5,8,38939.0,61609.0,10,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20240,94112,90,9,73104.0,57629.0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20241,94103,53,3,23016.0,31131.0,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20242,94103,32,1,23016.0,31131.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# scale features
data_scaler = StandardScaler()

In [19]:
# fit transform 
X_encoded_scaled = data_scaler.fit_transform(X_encoded)
X_encoded_scaled[:1]

array([[ 2.02549393,  1.42048832,  0.22407683, -0.50089233, -0.59322238,
         0.76038594, -0.01721836, -0.06057074, -0.0070285 , -0.0722064 ,
        -0.04667138, -0.17506744, -0.23129437, -0.20204609, -0.15129174,
        -0.01405804, -0.14466839, -0.05719169,  4.92360592, -0.14106823,
        -0.0521944 , -0.15299117, -0.11045361, -0.13682034, -0.00994004,
        -0.01859842, -0.25024265, -0.08726597, -0.14573248, -0.04925792,
        -0.05675536, -0.04504888, -0.10625365, -0.0952464 , -0.27507124,
        -0.14643794, -0.18155591, -0.00994004, -0.01217432, -0.26506923,
        -0.22445793, -0.05452203, -0.03065015, -0.01571775, -0.08464147,
        -0.02534913, -0.00994004, -0.06569717, -0.04772267, -0.02331665,
        -0.0660753 , -0.03978967, -0.13205989, -0.01405804, -0.02108966,
        -0.02723068, -0.01859842, -0.28552724, -0.02435405, -0.12470493,
        -0.21037762, -0.32388457, -0.17759899, -0.07081065, -0.03445205,
        -0.02723068, -0.21444189, -0.02812442]])

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'Low Risk': 7323, 'Moderate Risk': 5724, 'High Risk': 2136})

In [21]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [22]:
Counter(y_resampled)

Counter({'Low Risk': 7323, 'Moderate Risk': 7323, 'High Risk': 7323})

In [23]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

ValueError: could not convert string to float: 'Unclean or degraded floors walls or ceilings'